<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/Retina-VesselNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Retinal-VesselNet under Tensorflow2 [OneForAll]**
a segmentation model based on U-net for the task of retinal-vessel segmentation

**AUTHOR**: github @deeptrial

**PROJECT HOMEPAGE**: https://github.com/DeepTrial/Retina-VesselNet

**New Features**：

support tensorflow2<br>
support google colab<br>
more comprehensive notes<br>
more concise code<br>
#**[Part-I] Environment Preparation**
If you are using Colab, Please remember to switch GPU mode before the following steps<br>
如果在使用Google Colab，请确认已启用GPU加速

#**Check the GPU**

In [1]:
!nvidia-smi
import os
import datetime
os.environ['CUDA_VISIBLE_DEVICES'] = "1"     #use GPU-0 
%load_ext tensorboard

Thu Sep  2 09:34:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#**Check the version of Tensorflow**

In [2]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
gpus=tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2.6.0
[]


#**Install the requirement package**

In [3]:
!pip install tqdm
!pip install matplotlib
!pip install opencv-python
!pip install tf-nightly
!pip install scikit-learn

!pip install onnxruntime
!pip install -U tf2onnx

     |████████████████████████████████| 468.5 MB 19 kB/s 
     |████████████████████████████████| 1.3 MB 87.6 MB/s 
     |████████████████████████████████| 2.3 MB 73.0 MB/s 
     |████████████████████████████████| 5.7 MB 58.6 MB/s 
     |████████████████████████████████| 463 kB 83.6 MB/s 
     |████████████████████████████████| 12.8 MB 26.4 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.


     |████████████████████████████████| 4.5 MB 8.2 MB/s 
     |████████████████████████████████| 430 kB 8.3 MB/s 
     |████████████████████████████████| 12.3 MB 71.4 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.7.0.dev20210902 requires flatbuffers~=2.0, but you have flatbuffers 1.12 which is incompatible.


#**Data preparation**

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
#Extract dataset
!apt-get install rar
!rar x "/content/drive/MyDrive/Deep_learning/FundusPhoto/VesselNet.rar" "/content"


Reading package lists... Done
Building dependency tree       
Reading state information... Done
rar is already the newest version (2:5.5.0-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.

RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help


Extracting from /content/drive/MyDrive/Deep_learning/FundusPhoto/VesselNet.rar

Creating    /content/VesselNet                                        OK
Creating    /content/VesselNet/checkpoint                             OK
Extracting  /content/VesselNet/checkpoint/DRIVE_PR.png                     0%  OK 
Extracting  /content/VesselNet/checkpoint/DRIVE_ROC.png                    0%  OK 
Extracting  /content/VesselNet/checkpoint/events.out.tfevents.1540367701.DESKTOP-PGVTMOP       0%  OK 
Extracting  /conte